# Import dataset and clean the data

In [1]:
import pandas as pd

In [2]:
salesdata = pd.read_excel('LaptopSalesLL.xlsx')

First, we take a look at the dataset and have a general idea of each column.

In [3]:
salesdata.head()

,Date,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y,Customer Latitude,Customer Longitude,Store Latitude,Store Longitude
0,2008-01-01 00:01:19,163,EC4V 5BH,SE1 2BN,455.0,15,5,1,2.0,Yes,80,Yes,532041,180995,534057.0,179682.0,51.512579,-0.098731,51.500186,-0.070057
1,2008-01-01 00:02:52,320,SW4 0JL,SW12 9HD,545.0,15,6,1,2.0,No,300,No,529240,175537,528739.0,173080.0,51.464129,-0.140912,51.442082,-0.149046
2,2008-01-01 00:04:18,23,EC3V 1LR,E2 0RY,515.0,15,4,1,2.0,Yes,300,Yes,533095,181047,535652.0,182961.0,51.512699,-0.083435,51.529346,-0.045410
3,2008-01-01 00:04:40,169,SW1P 3AU,SE1 2BN,395.0,15,5,1,2.0,No,40,Yes,529902,179641,534057.0,179682.0,51.500787,-0.129897,51.500186,-0.070057
4,2008-01-01 00:06:04,365,EC4V 4EG,SW1V 4QQ,585.0,15,6,2,2.0,No,120,Yes,531684,180948,528924.0,178440.0,51.51212,-0.103737,51.490255,-0.144474


In [4]:
salesdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297364 entries, 0 to 297363
Data columns (total 20 columns):
Date                      297364 non-null datetime64[ns]
Configuration             297364 non-null int64
Customer Postcode         297364 non-null object
Store Postcode            297364 non-null object
Retail Price              283929 non-null float64
Screen Size (Inches)      297364 non-null int64
Battery Life (Hours)      297364 non-null int64
RAM (GB)                  297364 non-null int64
Processor Speeds (GHz)    297364 non-null float64
Integrated Wireless?      297364 non-null object
HD Size (GB)              297364 non-null int64
Bundled Applications?     297364 non-null object
customer X                297364 non-null int64
customer Y                297364 non-null int64
store X                   297216 non-null float64
store Y                   297216 non-null float64
Customer Latitude         297364 non-null object
Customer Longitude        297364 non-null object
St

The data type of 'Customer Latitude' and 'Customer Longitude' is not object but float, so we need to convert it for further use.

In [5]:
salesdata[['Customer Latitude','Customer Longitude']]=salesdata[['Customer Latitude',\
                                                'Customer Longitude']].apply(pd.to_numeric,errors = 'coerce')

Next, we check for the null values of each columns and drop the lines with missing values.

In [6]:
salesdata.isnull().sum(axis=0)

Date                          0
Configuration                 0
Customer Postcode             0
Store Postcode                0
Retail Price              13435
Screen Size (Inches)          0
Battery Life (Hours)          0
RAM (GB)                      0
Processor Speeds (GHz)        0
Integrated Wireless?          0
HD Size (GB)                  0
Bundled Applications?         0
customer X                    0
customer Y                    0
store X                     148
store Y                     148
Customer Latitude           384
Customer Longitude          384
Store Latitude              148
Store Longitude             148
dtype: int64

Because some cutomers visited more than one stores and we only care about where did thoes customers purchase the laptops in the end. So we think it is reasonable to remove the rows without pricing details. We compare the result with the one didn't remove these rows, the number of customers just dropped a little bit. And the number of the stores remains unchanged.

In [7]:
sales_clean=salesdata.dropna()
sales_clean.shape

(283418, 20)

Also, we compute the average of unique store coordinates to set the map center.

In [8]:
central = []
central.append(sales_clean['Store Latitude'].unique().mean())
central.append(sales_clean['Store Longitude'].unique().mean())
central

[51.50106773333333, -0.1345508]

# Draw the map with the package folium

## create map

In [9]:
import folium

m = folium.Map(location = central, zoom_start=10)
m

## Create the store locations

Create a dataframe to save distinct store locations and draw those points as red markers on the map. <br>When click on the marker, show the store information.

In [10]:
store=sales_clean.groupby(['Store Latitude', 'Store Longitude','Store Postcode'])['Retail Price'].\
                            agg(['mean', 'count']).\
                            reset_index()
store.head()

,Store Latitude,Store Longitude,Store Postcode,mean,count
0,51.398191,-0.093631,CR7 8LE,471.843580,6604
1,51.418947,-0.305625,KT2 5AU,521.473575,4333
2,51.442082,-0.149046,SW12 9HD,521.212815,12673
3,51.461802,-0.199785,SW18 1NN,520.357168,14069
4,51.483307,-0.025775,SE8 3JD,520.680215,16583


## Draw the customer locations

Similarly, we create a dataframe to save distinct customer locations and draw those points as blue circles on the map.<br> When click on the circle, show the customer information.

In [11]:
customer=sales_clean.groupby(['Customer Latitude', 'Customer Longitude','Customer Postcode'])['Retail Price'].\
                            agg(['mean', 'count']).\
                            reset_index()
customer.head()

,Customer Latitude,Customer Longitude,Customer Postcode,mean,count
0,51.367827,-0.103900,CR0 4HA,498.926282,312
1,51.368895,-0.099891,CR0 1NF,512.472669,311
2,51.371064,-0.100562,CR0 1NA,498.886905,336
3,51.372037,-0.100091,CR0 1QD,508.258160,337
4,51.372889,-0.101593,CR0 1RG,514.370717,321


In [12]:
for eachrow in customer.values:
    customer_info='Customer postcode is {}.<br>Average spending is {}.<br>Number of purchase is {}.'\
            .format(eachrow[2],round(eachrow[3],2),eachrow[4])
    folium.Circle(location = eachrow[0:2], color='blue', \
                  fill=True, fill_color='blue',fill_opacity=1,radius= 10,\
                  popup = customer_info ).add_to(m)
m

## Draw the Store markers and lines

To avoid repetitively drawing the lines, we drop the duplicates of the sales data from 283418 to 2499 observations, which greatly improves the speed.

In [13]:
sales_uniq=sales_clean.loc[:,'Customer Latitude':'Store Longitude'].drop_duplicates()
sales_uniq.shape

(2499, 4)

Then we draw the lines of all the transactions and add every lines of one store to its feature group for the following interactive function. To satisfy our click-show-line objecive, we set the orginal line to be transaparent.

In [14]:
#m.save('GroupProject.html')

from folium import FeatureGroup

    

for eachrow in store.values:
    #create a feature group for each store
    store_lines=folium.FeatureGroup(name = eachrow[2],show=True)
    
    #add each store marker to the each featuregroup
    store_info='Store postcode is {}.<br>Average retail price is {}.<br>Number of transactions is {}.'\
            .format(eachrow[2],round(eachrow[3],2),eachrow[4])
    folium.Marker(location = eachrow[0:2], icon=folium.Icon(color='red',icon='laptop',prefix='fa'), \
                  popup = store_info).add_to(store_lines)


    #subset the sales dataset of each store
    each_store_sales=sales_uniq[sales_uniq.apply(lambda x : x['Store Latitude'] == eachrow[0] \
                                      and x['Store Longitude'] == eachrow[1], axis=1)]
    
    #add all the lines connected with each store to its featuregroup
    all_points=[]
    for each in each_store_sales.values:
        points=[]
        each_customer=tuple(each[0:2])
        each_store=tuple(each[2:])
        points.append(each_customer)
        points.append(each_store) #points is a list of two tuples 
        all_points.append(points)
    folium.PolyLine(all_points, color="red", weight=1, opacity=0).add_to(store_lines)  
    store_lines.add_to(m)
m

Because we will use click to 'show' the lines, so we don't need layercontol anymore

In [15]:
#add layer control
#from folium.map import LayerControl
#LayerControl().add_to(m)
#m

In [16]:
m.save('GroupProject.html')

# Add interactive function

We cannot easily add onclick event to the map in the package folium. <br> But since folium does create LeafletJS code, we can modify the resulting html to make it work.

In [17]:
import re
import fileinput

with open("GroupProject.html") as inf:
       txt = inf.read()

polylines = re.findall(r'\bpoly_line_\w+', txt)
polylines = sorted(set(polylines),key=polylines.index)
#sorted(list, key=list.index) will give us the unique values in its original order
#So the functions of click can be inserted in the coressponding order


groups = re.findall(r'\bfeature_group_\w+', txt)
groups = sorted(set(groups),key=groups.index)

#Set global varible will cause some incovenience:
#Like when we clicked one store, we had to click two times to show the lines of another store 
#without 'removing' the previous ones.
#So, we set count varible for each store.

count_list =['count_{}'.format(i) for i in range(15)]


In [18]:
%%time

for i in range(len(polylines)):
    pattern = "                    .addTo({});\n".format(groups[i])
    variable = "var {} =1;\n".format(count_list[i])
    pattern2 = groups[i] + ".on('click',function(e)"+ '''{
    var layer = e.target;'''+ \
    '{}={}+1'.format(count_list[i],count_list[i],count_list[i])+ \
    '\nif({}%2==0)'.format(count_list[i])+'''{
    layer.setStyle({
        color: 'red',
        opacity: 1,
        weight: 1
    });}
    else{
    layer.setStyle({
        color: 'red',
        opacity: 0,
        weight: 1
    });
    }       
});\n\n'''


    #pattern3 = groups[i]+ '''.off('click');'''
    # when mouseout we set the color of lines to be transparant.
    
    with open("GroupProject.html") as inf:
       txt = inf.read()
    for linenum,line in enumerate( fileinput.FileInput("GroupProject.html",inplace=1) ):    
        if pattern in line:
           print(line.rstrip())
           print(variable)
           print(pattern2)
           #print(pattern3)
        else:
           print(line.rstrip())


CPU times: user 530 ms, sys: 64.4 ms, total: 595 ms
Wall time: 610 ms


add hoverover event to show the summary of customer information

In [19]:
# this code will take you about 40s to run.
# hover popup
# hover over customers to show purchase summary.
import re
import fileinput

with open("GroupProject.html") as inf:
       txt = inf.read()

circles = re.findall(r'\bcircle_\w+', txt)
circles = list(set(circles))

for i in range(len(circles)):
    pattern = circles[i] + ".bindPopup"
    pattern2 = circles[i] + ".on('mouseover', function (e) {this.openPopup();});"
    pattern3 = circles[i] + ".on('mouseout', function (e) {this.closePopup();});"
    for linenum,line in enumerate( fileinput.FileInput("GroupProject.html",inplace=1) ):    
        if pattern in line:
           print(line.rstrip())
           print(pattern2)
           print(pattern3)
        else:
           print(line.rstrip())

add hoverover event to show the summary of store information

In [20]:
# hover popup
# hover over stores to show store summary

with open("GroupProject.html") as inf:
       txt = inf.read()

markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for i in range(len(markers)):
    pattern = markers[i] + ".bindPopup"
    pattern2 = markers[i] + ".on('mouseover', function (e) {this.openPopup();});"
    pattern3 = markers[i] + ".on('mouseout', function (e) {this.closePopup();});"
    with open("GroupProject.html") as inf:
       txt = inf.read()
    for linenum,line in enumerate( fileinput.FileInput("GroupProject.html",inplace=1) ):    
        if pattern in line:
           print(line.rstrip())
           print(pattern2)
           print(pattern3)
        else:
           print(line.rstrip())

In [3]:
#the code can show save our final html parser to the file scriptofmap.txt. 

#with open("scriptofmap.txt", "w") as text_file:
#    text_file.write(txt)

* Our final work is able be interacted with in the html file that attached in the zip file uploaded.